In [51]:
import pandas as pd
import logging
import numpy as np
from icecream import ic
from itertools import product
from geopy.distance import geodesic
from dataclasses import dataclass
import functools
import random
from tqdm import tqdm

logging.basicConfig(level=logging.DEBUG)

In [52]:
PATH = "./cities/"
INSTANCE = "russia.csv"

In [53]:
cities = pd.read_csv(f"{PATH}{INSTANCE}", header=None, names=["City", "lat", "long"])
#cities

In [54]:
#show only first column
cities_names = np.array([c['City'] for _,c in cities.iterrows()])
#cities_names

In [55]:
coordinates = cities[["lat","long"]].to_numpy()
#coordinates

## Helper funciton

In [56]:
def distance(c1,c2):
    return geodesic(c1,c2).km

#distance(coordinates[0],coordinates[1])

In [57]:
dist_matrix = np.array([[distance(c1,c2) for c1 in coordinates] for c2 in coordinates])

In [58]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper


@counter
def tsp_cost(tsp):
#    ic(tsp[0], tsp[-1])
#    assert tsp[0] == tsp[-1]
#    assert set(tsp) == set(range(len(cities)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += dist_matrix[c1, c2]
    return tot_cost

In [59]:
def greedy_tsp():
    visited = [False]*len(coordinates)
    dist = dist_matrix.copy()

    city = 0
    tsp = list()
    #ic(dist)
    tsp.append(city)

    while not np.all(visited):
        min_dist = np.inf
        next_city = None
        for i in range(len(coordinates)):
            if not visited[i] and dist[city][i] < min_dist and i != city:
                next_city = i
                min_dist = dist[city][i]
        visited[next_city] = True
        tsp.append(next_city)
        #ic("visiting city: ", cities_names[city])
       # logging.debug(
           # f"step: {cities_names[city]} -> {cities_names[next_city]} ({min_dist:.2f}km)")
        city = next_city
        
                

    tot_cost = 0
   # for c1, c2 in zip(tsp, tsp[1:]):
    #    tot_cost += dist_matrix[c1, c2]
    #logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tot_cost:.2f}km")  
    return tsp

## Greedy with random starting point

In [60]:
def greedy_tsp_random_start(start):
    visited = [False]*len(coordinates)
    dist = dist_matrix.copy()

    city = start
    visited[city] = True

    tsp = list()
    #ic(dist)
    tsp.append(city)

    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        # logging.debug(
        #     f"step: {cities_names[city]} -> {cities_names[closest]} ({dist[city,closest]:.2f}km)")
       
        visited[closest] = True
        city = closest
        tsp.append(int(city))
  
    # logging.debug(
    #     f"step: {cities_names[tsp[-1]]} -> {cities_names[tsp[0]]} ({dist[tsp[0],tsp[-1]]:.2f}km)")
    tsp.append(tsp[0])
    
            
    tot_cost =tsp_cost(tsp)
    #logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tot_cost:.2f}km")  
    return tsp

In [61]:
""" best_sol = -1
best_cost = np.inf

for i in range(len(cities)):
    sol = greedy_tsp_random_start(i)
    cost = tsp_cost(sol)
    # logging.info(f"initial_sol: {sol}")
    # logging.info(f"initial_cost: {cost}")
    if cost < best_cost:
        best_sol = sol
        best_cost = cost

logging.info(f"best_sol: {best_sol}")
logging.info(f"best_cost: {best_cost}") """

' best_sol = -1\nbest_cost = np.inf\n\nfor i in range(len(cities)):\n    sol = greedy_tsp_random_start(i)\n    cost = tsp_cost(sol)\n    # logging.info(f"initial_sol: {sol}")\n    # logging.info(f"initial_cost: {cost}")\n    if cost < best_cost:\n        best_sol = sol\n        best_cost = cost\n\nlogging.info(f"best_sol: {best_sol}")\nlogging.info(f"best_cost: {best_cost}") '

## gready + EA

### helper functions

In [62]:
@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None

def fitness(individual):
    return  -float(tsp_cost(individual.genome))

def parent_selection(population):
    candidates = sorted(np.random.choice(population, 2), key=lambda e: e.fitness, reverse=True)
    return candidates[0]


In [63]:
def pmover(p1, p2):
    i1 = np.random.randint(len(p1))
    i2 = np.random.randint(len(p1))
        # i1 = 3
    # i2 = 6
    while i1 == i2:
        i2 = np.random.randint(len(p1))
  #  ic(i1, i2)

    if i1 > i2:
        i1, i2 = i2, i1
 

    o1 = p1.copy()
    o2 = p2.copy()
    o1[i1:i2] = p2[i1:i2]
    o2[i1:i2] = p1[i1:i2]

    for i in range(0, i1):
        while o1[i] in o1[i1:i2]:
            o1[i] = p1[p2.index(o1[i])]
        while o2[i] in o2[i1:i2]:
            o2[i] = p2[p1.index(o2[i])]

    for i in range(i2, len(o1)):
        while o1[i] in o1[i1:i2]:
            o1[i] = p1[p2.index(o1[i])]
        while o2[i] in o2[i1:i2]:
            o2[i] = p2[p1.index(o2[i])]
    
   # ic(p1,p2)
   # ic(o1,o2)
    return o1, o2

def pmover_2(p1, p2):
    # Randomly select two indices and sort them to define the crossover segment
    i1, i2 = sorted(np.random.choice(len(p1), 2, replace=False))

    # Create copies for the offspring
    o1, o2 = p1.copy(), p2.copy()

    # Perform crossover in the segment between i1 and i2
    o1[i1:i2], o2[i1:i2] = p2[i1:i2], p1[i1:i2]

    # Create maps to track replacements for each offspring
    map_o1, map_o2 = {}, {}

    # Initialize mapping dictionaries for the crossover segment
    for i in range(i1, i2):
        map_o1[p2[i]] = p1[i]
        map_o2[p1[i]] = p2[i]

    # Function to trace mappings to avoid duplicates
    def resolve_conflict(value, mapping):
        while value in mapping:
            value = mapping[value]
        return value

    # Resolve conflicts for elements outside the crossover segment
    for i in range(len(p1)):
        if i < i1 or i >= i2:  # Only for elements outside the crossover
            o1[i] = resolve_conflict(o1[i], map_o1)
            o2[i] = resolve_conflict(o2[i], map_o2)

    return o1, o2

In [ ]:
test1 = greedy_tsp_random_start(0)
test2 = greedy_tsp_random_start(1)
for i in range(1000):
    test1, test2 = pmover(test1, test2)


In [ ]:
test1 = greedy_tsp_random_start(0)
test2 = greedy_tsp_random_start(1)
for i in range(1000):
    test1, test2 = pmover_2(test1, test2)


In [64]:
def inv_mutation(individual):
    mutated_genome = individual.genome[:-1]
 
     # Select two random indices for the inversion segment
    i, j = sorted(np.random.randint(0, len(individual.genome), 2))
    while i == j:
        i, j = sorted(np.random.randint(0, len(individual.genome), 2))
   # ic(i, j)
    mutated_genome[i:j+1] = mutated_genome[i:j+1][::-1]

    return mutated_genome
    
def generate_individuals(parent:Individual)->Individual:
    new_genome = random.sample(parent.genome[:-1],len(parent.genome[:-1]))
    new_genome.append(new_genome[0])
    return Individual(new_genome)

### inverse mutation or partially mapped crossover

In [65]:
""" POPULATION_SIZE = int( len(cities)*1.5)
population = list()

# generate random initial individual using a greedy starting from a random city
initial_individual = Individual(greedy_tsp_random_start(np.random.randint(0, len(cities))))
#initial_individual = Individual(greedy_tsp())
ic(tsp_cost(initial_individual.genome))

# generate random initial population starting from the initial individual
population = [generate_individuals(initial_individual) for _ in range(POPULATION_SIZE-1)]
# add the initial individual
population.append(initial_individual)


population.sort(key=lambda i: tsp_cost(i.genome))
ic(tsp_cost(population[0].genome))


for i in population:
    i.fitness = -tsp_cost(i.genome)
    

OFFSPRING_SIZE = int(POPULATION_SIZE*0.25)
MAX_GENERATIONS = 10000

for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):

        if np.random.random() < 0.5:
            p = parent_selection(population)
            o = inv_mutation(p)
            o.append(o[0])
            o = Individual(o)            
            offspring.append(o)
            
        else :

            p1 = parent_selection(population)
            p2 = parent_selection(population)

            o1,o2 = pmover(p1.genome[:-1], p2.genome[:-1])

            o1.append(o1[0])
            o2.append(o2[0])

            o1,o2 = Individual(o1), Individual(o2)

            offspring.append(o1)
            offspring.append(o2)

    for i in offspring:
        i.fitness = fitness(i)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]


for i in population:
    #add last one
    i.genome = np.append(i.genome, i.genome[0])
    i.fitness = -tsp_cost(i.genome)

population.sort(key=lambda i: i.fitness, reverse=True)
ic(-population[0].fitness, tsp_cost.calls) """

' POPULATION_SIZE = int( len(cities)*1.5)\npopulation = list()\n\n# generate random initial individual using a greedy starting from a random city\ninitial_individual = Individual(greedy_tsp_random_start(np.random.randint(0, len(cities))))\n#initial_individual = Individual(greedy_tsp())\nic(tsp_cost(initial_individual.genome))\n\n# generate random initial population starting from the initial individual\npopulation = [generate_individuals(initial_individual) for _ in range(POPULATION_SIZE-1)]\n# add the initial individual\npopulation.append(initial_individual)\n\n\npopulation.sort(key=lambda i: tsp_cost(i.genome))\nic(tsp_cost(population[0].genome))\n\n\nfor i in population:\n    i.fitness = -tsp_cost(i.genome)\n    \n\nOFFSPRING_SIZE = int(POPULATION_SIZE*0.25)\nMAX_GENERATIONS = 10000\n\nfor g in tqdm(range(MAX_GENERATIONS)):\n    offspring = list()\n    for _ in range(OFFSPRING_SIZE):\n\n        if np.random.random() < 0.5:\n            p = parent_selection(population)\n            o 

### inverse mutation + partially mapped crossover

In [66]:
POPULATION_SIZE = int( len(cities)*1.5)
population = list()

# generate random initial individual using a greedy starting from a random city
initial_individual = Individual(greedy_tsp_random_start(np.random.randint(0, len(cities))))
#initial_individual = Individual(greedy_tsp())
ic(tsp_cost(initial_individual.genome))

# generate random initial population starting from the initial individual
population = [generate_individuals(initial_individual) for _ in range(POPULATION_SIZE-1)]
# add the initial individual
population.append(initial_individual)


population.sort(key=lambda i: tsp_cost(i.genome))
ic(tsp_cost(population[0].genome))


for i in population:
    i.fitness = -tsp_cost(i.genome)
    

OFFSPRING_SIZE = int(POPULATION_SIZE*0.25)
MAX_GENERATIONS = 10_000

for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):

        """if np.random.random() < 0.5:
            p = parent_selection(population)
            o = inv_mutation(p)
            o.append(o[0])
            o = Individual(o)            
            offspring.append(o)
            
        else : """

        p1 = parent_selection(population)
        p2 = parent_selection(population)
        
        inv_p1 = inv_mutation(p1)
        inv_p1.append(inv_p1[0])
        inv_p2 = inv_mutation(p2)
        inv_p2.append(inv_p2[0])
        
        o1,o2 = pmover_2(inv_p1[:-1], inv_p2[:-1])

#        o1,o2 = pmover(p1.genome[:-1], p2.genome[:-1])
        o1.append(o1[0])
        o2.append(o2[0])

        o1,o2 = Individual(o1), Individual(o2)

        offspring.append(o1)
        offspring.append(o2)

    for i in offspring:
        i.fitness = fitness(i)
        # ic(i.genome)
        # ic(i.fitness)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
#    ic("best sol: ",-population[0].fitness)



for i in population:
    #add last one
    i.genome = np.append(i.genome, i.genome[0])
    i.fitness = -tsp_cost(i.genome)

population.sort(key=lambda i: i.fitness, reverse=True)
ic(-population[0].fitness, tsp_cost.calls)

ic| tsp_cost(initial_individual.genome): np.float64(45201.66805065893)
ic| tsp_cost(population[0].genome): np.float64(45201.66805065893)
100%|██████████| 10000/10000 [05:50<00:00, 28.55it/s]
ic| -population[0].fitness: np.float64(35094.55710687571)
    tsp_cost.calls: 1240753


(np.float64(35094.55710687571), 1240753)